# Replication of 'Optimal Portfolio Choice with Estimation Risk'

Kan, R., Wang, X., & Zhou, G. (2021). Optimal Portfolio Choice with Estimation Risk: No Risk-Free Asset Case. Management Science. https://doi.org/10.1287/mnsc.2021.3989


## 1. Porfolio construct

Target portfolio - optimal combining portfolio with shrunk covariance matrix

\begin{equation}
\hat{w}_{q, t}^{L W 2004}=\hat{w}_{g, t}^{L W 2004}+\frac{g_{3}\left(\hat{\psi}_{t}^{L}\right)}{\gamma} \hat{w}_{z, t}^{L W 2004}
\end{equation}

Benchmark portfolio - optimal combining portfolio

\begin{equation}
\hat{w}_{q, t}=\hat{w}_{t}\left(\hat{c}_{t}\right)=\hat{w}_{g, t}+\frac{g_{3}\left(\hat{\psi}_{t}^{2}\right)}{\gamma} \hat{w}_{z, t}
\end{equation}

Where,

\begin{align}
\hat{w}_{g, t}^{L W 2004} &=\frac{\left(\hat{\Sigma}_{t}^{L W 2004}\right)^{-1} 1_{N}}{1_{N}^{\prime}\left(\hat{\Sigma}_{t}^{L W 2004}\right)^{-1} 1_{N}} \\

\hat{w}_{g, t} &=\frac{\hat{\Sigma}_{t}^{-1} 1_{N}}{1_{N}^{\prime} \hat{\Sigma}_{t}^{-1} 1_{N}}\\

\hat{w}_{z, t}^{L W 2004}&=\left(\hat{\Sigma}_{t}^{L W 2004}\right)^{-1}\left[\hat{\mu}_{t}-1_{N}\left(\hat{\mu}_{t}^{\prime} \hat{w}_{g, t}^{L W 2004}\right)\right]\\

\hat{w}_{z, t}&=\hat{\Sigma}_{t}^{-1}\left(\hat{\mu}_{t}-1_{N} \hat{\mu}_{g, t}\right)\\

\hat{c}_{t}&=g_{3}\left(\hat{\psi}_{t}^{2}\right)=\frac{k \hat{\psi}_{a, t}^{2}}{\hat{\psi}_{a, t}^{2}+\frac{N-1}{h}}\\

\hat{\psi}_{a, t}^{2}&=\frac{(h-N-1) \hat{\psi}_{t}^{2}-(N-1)}{h}+\frac{2\left(\hat{\psi}_{t}^{2}\right)^{\frac{N-1}{2}}\left(1+\hat{\psi}_{t}^{2}\right)^{-\frac{h-2}{2}}}{h \mathrm{~B}_{\hat{\psi}_{t}^{2} /\left(1+\hat{\psi}_{t}^{2}\right)}((N-1) / 2,(h-N+1) / 2)}\\

\hat{\psi}_{t}^{2}&=\hat{\mu}_{t}^{\prime} \hat{\Sigma}_{t}^{-1} \hat{\mu}_{t}-\frac{\left(1_{N}^{\prime} \hat{\Sigma}_{t}^{-1} \hat{\mu}_{t}\right)^{2}}{\left(1_{N}^{\prime} \hat{\Sigma}_{t}^{-1} 1_{N}\right)}=\frac{z_{2}^{2}+u_{0}}{v_{2}}\\

\hat{\Sigma}_{t}^{L W 2004}&=\left(1-\rho_{t}\right) \hat{\Sigma}_{t}+\rho_{t} v_{t} I_{N}\\

\hat{\Sigma}_{t}&=\frac{1}{h} \sum_{s=t-h+1}^{t}\left(r_{s}-\hat{\mu}_{t}\right)\left(r_{s}-\hat{\mu}_{t}\right)^{\prime} \\

\hat{\mu}_{t}&=\frac{1}{h} \sum_{s=t-h+1}^{t} r_{s}\\

\hat{\mu}_{g, t}&=\left(1_{N}^{\prime} \hat{\Sigma}_{t}^{-1} \hat{\mu}_{t}\right) /\left(1_{N}^{\prime} \hat{\Sigma}_{t}^{-1} 1_{N}\right)\\

k & =\frac{(h-N)(h-N-3)}{h(h-2)}\\

\rho_{t} & =\frac{\operatorname{Min}\left[\frac{1}{h^{2}} \sum_{s=t-h+1}^{t}\left\|\left(r_{s}-\hat{\mu}_{t}\right)\left(r_{s}-\hat{\mu}_{t}\right)^{\prime}-\hat{\Sigma}_{t}\right\|^{2},\left\|\hat{\Sigma}_{t}-v_{t} I_{N}\right\|^{2}\right]}{\left\|\hat{\Sigma}_{t}-v_{t} I_{N}\right\|^{2}}\\

\end{align}


In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

import matplotlib.pyplot as plt
from numpy.linalg import inv
from scipy.special import betainc

import sys
sys.path.append('/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/codes/')

from module.data_library import data_library
from module.portfolio_generator import portfolio_generator
from module.vectorized_backtesting import vectorized_backtesting, get_portfolio_performance
from module.portfolio_analyser import get_sharpe_ratio, get_turnover
from module.tool_box import export_dataframe_to_latex_table
from module.portfolio_library import get_naive_combine_portfolio, get_gmv, get_zero_invest_portfolio, get_opt_combine_portfolio

## 2. Implementation of different portfolios

### 2.1 Zero-investiment portfolio

\begin{align}
\hat{\boldsymbol{w}}_{z, t} &=\hat{\Sigma}_{t}^{-1}\left(\hat{\mu}_{t}-\mathbf{1}_{N} \hat{\mu}_{g, t}\right) \\
\mu_g &= \mathbf{1}_{N}^{\prime} \Sigma^{-1} \mu /\left(\mathbf{1}_{N}^{\prime} \Sigma^{-1} \mathbf{1}_{N}\right) \\
\hat{\Sigma}_{t}&=\frac{1}{h} \sum_{s=t-h+1}^{t}\left(r_{s}-\hat{\mu}_{t}\right)\left(r_{s}-\hat{\mu}_{t}\right)^{\prime} \\
\hat{\mu}_{t}&=\frac{1}{h} \sum_{s=t-h+1}^{t} r_{s}
\end{align}

In [ ]:
def get_zero_invest_portfolio(return_df: pd.DataFrame) -> np.array:
    '''
    Calculate the zero investment portfolio using given period data.

    Args:
        return_df: stock returns dataframe

    Returns:
        w_z: portfolio weights
    '''
    return_matrix = return_df.values
    sigma = np.cov(return_matrix, rowvar= False)
    mu = np.mean(return_matrix, axis=0, keepdims=True).T
    sigma_inv = inv(sigma)
    I = np.ones((len(mu), 1))
    mu_gmv = I.T @ sigma_inv @ mu / (I.T @ sigma_inv @ I)
    w_z = sigma_inv @ (mu - I @ mu_gmv)

    return w_z

### 2.2 Naive combing portfolio
The naive combing portfolio is calculated as following
$$
\hat{\boldsymbol{w}}_{p, t}=\hat{\boldsymbol{w}}_{g, t}+\frac{1}{\gamma} \hat{\boldsymbol{w}}_{z, t}
$$


In [90]:
def get_naive_combine_portfolio(
    return_df: pd.DataFrame,
    gamma: int = 3) -> np.array:
    '''
    Calculate the combine portfolio where don't take into account the estimation risk.
    In this portfolio, it combines GMV and zero investiment portfolio according to given gamma.
    It doesn't optimize the combining ratio.

    Args:
        return_df: stock return matrix
        gamma: risk aversion coefficient

    Returns:
        w_p: portfolio weight 
    '''
    c = 1
    w_z = get_zero_invest_portfolio(return_df)
    w_g = get_gmv(return_df)
    w_p = w_g + (c / gamma) * w_z

    return w_p

## 2.3 Opt combing portfolio
\begin{equation}
\hat{w}_{q, t}=\hat{w}_{t}\left(\hat{c}_{t}\right)=\hat{w}_{g, t}+\frac{g_{3}\left(\hat{\psi}_{t}^{2}\right)}{\gamma} \hat{w}_{z, t}
\end{equation}
Where,

\begin{align}
\hat{c}_{t}&=g_{3}\left(\hat{\psi}_{t}^{2}\right)=\frac{k \hat{\psi}_{a, t}^{2}}{\hat{\psi}_{a, t}^{2}+\frac{N-1}{h}}\\

\hat{\psi}_{a, t}^{2}&=\frac{(h-N-1) \hat{\psi}_{t}^{2}-(N-1)}{h}+\frac{2\left(\hat{\psi}_{t}^{2}\right)^{\frac{N-1}{2}}\left(1+\hat{\psi}_{t}^{2}\right)^{-\frac{h-2}{2}}}{h \mathrm{~B}_{\hat{\psi}_{t}^{2} /\left(1+\hat{\psi}_{t}^{2}\right)}((N-1) / 2,(h-N+1) / 2)}\\

\hat{\psi}_{t}^{2}&=\hat{\mu}_{t}^{\prime} \hat{\Sigma}_{t}^{-1} \hat{\mu}_{t}-\frac{\left(1_{N}^{\prime} \hat{\Sigma}_{t}^{-1} \hat{\mu}_{t}\right)^{2}}{\left(1_{N}^{\prime} \hat{\Sigma}_{t}^{-1} 1_{N}\right)}=\frac{z_{2}^{2}+u_{0}}{v_{2}}\\

k&=\frac{(h-N)(h-N-3)}{h(h-2)}
\end{align}

$\hat{\mu}_t$ is the vector of stock mean return 

h is the size of rolling window

N is the number of assets


In [22]:
def get_opt_combine_portfolio(
    return_df: pd.DataFrame,
    rolling_period: int,
    gamma: int = 3) -> np.array:
    '''
    Calculate the optimal combine portfolio where don't take into account the estimation risk.
    In this portfolio, it combines GMV and zero investiment portfolio according to given gamma and intensity c.
    The c is the optimal combining ratio that that maximizes the ex- pected out-of-sample utility.

    Args:
        return_df: stock return matrix
        gamma: risk aversion coefficient

    Returns:
        w_p: portfolio weight 
    '''
    return_matrix = return_df.values
    h = rolling_period
    N = return_df.shape[1]
    sigma = np.cov(return_matrix, rowvar= False)
    mu = np.mean(return_matrix, axis=0, keepdims=True).T
    sigma_inv = inv(sigma)
    I = np.ones((N, 1))

    psi = mu.T @ sigma_inv @ mu - (I.T @ sigma_inv @ mu) ** 2 / (I.T @ sigma_inv @ I)
    psi_a = ((h - N - 1) * psi - (N - 1)) / h + \
    (2 * psi ** ((N - 1) / 2) * (1 + psi) ** (- (h - 2) / 2)) / (h * betainc((N - 1) / 2, (h - N + 1) / 2, psi / (1 + psi)))
    k = (h - N) * (h - N -3) / (h * (h - 2))

    c = (psi_a * k) / (psi_a + (N - 1) / h)
    w_z = get_zero_invest_portfolio(return_df)
    w_g = get_gmv(return_df)
    w_p = w_g + (c / gamma) * w_z

    return w_p

## 4. Backtest with industry, momentum and B/M portfolio

## 4.1 Backtesting to get portfolio performance 
I backtest two portfolio strategies, 1/N and GMV, on three different data sets, industry, momentum and Book to market.

The portfolio performance are reported in two measurement, sharpe ratio and turnover.

At the end, the calculated results are compared with benchmark results.

In [21]:
backtesting_setting = {
                        'data_frequncy': 'monthly',
                        'data_name':'industry',
                        'start':'1969-07-01', 
                        'end':'2018-12-01',
                        'rebalance_frequncy':'month',
                        'get_portfolio': get_opt_combine_portfolio,
                        'rolling_period':120
                      }
industry_opt_comb_performance = get_portfolio_performance(**backtesting_setting)
industry_opt_comb_performance

,industry
Sharpe ratio,0.056786
Turnover,2.432877
Sharpe ratio net,-0.016501


In [12]:
backtesting_setting = {
                        'data_frequncy': 'monthly',
                        'data_name':'B/M',
                        'start':'1927-01-01', 
                        'end':'2018-12-01',
                        'rebalance_frequncy':'month',
                        'get_portfolio': get_naive_combine_portfolio,
                        'rolling_period':120
                      }
bm_naive_comb_performance = get_portfolio_performance(**backtesting_setting)
bm_naive_comb_performance

,B/M
Sharpe ratio,0.203703
Turnover,24.605052
Sharpe ratio net,-0.003811


In [13]:
backtesting_setting = {
                        'data_frequncy': 'monthly',
                        'data_name':'moment',
                        'start':'1927-01-01', 
                        'end':'2018-12-01',
                        'rebalance_frequncy':'month',
                        'get_portfolio': get_naive_combine_portfolio,
                        'rolling_period':120
                      }
moment_naive_comb_performance = get_portfolio_performance(**backtesting_setting)
moment_naive_comb_performance

,moment
Sharpe ratio,0.224982
Turnover,4.966185
Sharpe ratio net,0.168338


## 4.2 Sharpe ratio compare
Gather shapre ratio result from the combination of two strategies and three different data sets.

Compare shapre ratio performance between calculated result and benchmark results.

In [11]:
performance_option = 'Sharpe ratio'
sr_dict = {
    'Momentum(10)':[moment_equal_performance.loc[performance_option].values[0],
                    moment_gmv_performance.loc[performance_option].values[0]],
    'Size-B/M(25)':[bm_equal_performance.loc[performance_option].values[0], 
                    bm_gmv_performance.loc[performance_option].values[0]],
    'Industry(49)':[industry_equal_performance.loc[performance_option].values[0], 
                    industry_gmv_performance.loc[performance_option].values[0]]
}
sr_df = pd.DataFrame(sr_dict, index=['1/N', 'W_g,t'])
sr_df

,Momentum(10),Size-B/M(25),Industry(49)
1/N,0.128126,0.148337,0.153371
"W_g,t",0.177097,0.199044,0.123324


In [12]:
sr_bench_dict = {
    'Momentum(10)':[0.1276, 0.1823],
    'Size-B/M(25)':[0.1484, 0.2085],
    'Industry(49)':[0.1527, 0.1227]
}
sr_bench_df = pd.DataFrame(sr_bench_dict, index=['1/N', 'W_g,t'])
sr_bench_df

,Momentum(10),Size-B/M(25),Industry(49)
1/N,0.1276,0.1484,0.1527
"W_g,t",0.1823,0.2085,0.1227


In [13]:
sr_diff_df = sr_df - sr_bench_df
sr_diff_df

,Momentum(10),Size-B/M(25),Industry(49)
1/N,0.000526,-0.000063,0.000671
"W_g,t",-0.005203,-0.009456,0.000624


## 4.3 Turnover compare

In [14]:
performance_option = 'Turnover'
turnover_dict = {
    'Momentum(10)':[moment_equal_performance.loc[performance_option].values[0],
                    moment_gmv_performance.loc[performance_option].values[0]],
    'Size-B/M(25)':[bm_equal_performance.loc[performance_option].values[0], 
                    bm_gmv_performance.loc[performance_option].values[0]],
    'Industry(49)':[industry_equal_performance.loc[performance_option].values[0], 
                    industry_gmv_performance.loc[performance_option].values[0]]
}
turnover_df = pd.DataFrame(turnover_dict, index=['1/N', 'W_g,t'])
turnover_df

,Momentum(10),Size-B/M(25),Industry(49)
1/N,0.016587,0.017179,0.031936
"W_g,t",0.272001,0.755546,0.806861


In [15]:
turnover_bench_dict = {
    'Momentum(10)':[0.0176, 0.2770],
    'Size-B/M(25)':[0.0182, 0.7665],
    'Industry(49)':[0.0341, 0.8227]
}
turnover_bench_df = pd.DataFrame(turnover_bench_dict, index=['1/N', 'W_g,t'])
turnover_bench_df

,Momentum(10),Size-B/M(25),Industry(49)
1/N,0.0176,0.0182,0.0341
"W_g,t",0.2770,0.7665,0.8227


In [16]:
turnover_diff_df = turnover_df - turnover_bench_df
turnover_diff_df

,Momentum(10),Size-B/M(25),Industry(49)
1/N,-0.001013,-0.001021,-0.002164
"W_g,t",-0.004999,-0.010954,-0.015839


## 4.4 Transanction cost
The portfolio returns net of proportional transaction costs is calculated as
$$
p_{t}=\left(1+\tilde{p}_{t}\right)\left(1-c \times \text { Turnover }_{t-1}\right)-1
$$

In [17]:
performance_option = 'Sharpe ratio net'
sr_net_dict = {
    'Momentum(10)':[moment_equal_performance.loc[performance_option].values[0],
                    moment_gmv_performance.loc[performance_option].values[0]],
    'Size-B/M(25)':[bm_equal_performance.loc[performance_option].values[0], 
                    bm_gmv_performance.loc[performance_option].values[0]],
    'Industry(49)':[industry_equal_performance.loc[performance_option].values[0], 
                    industry_gmv_performance.loc[performance_option].values[0]]
}
sr_net_df = pd.DataFrame(sr_net_dict, index=['1/N', 'W_g,t'])
sr_net_df

,Momentum(10),Size-B/M(25),Industry(49)
1/N,0.127457,0.147714,0.151994
"W_g,t",0.164556,0.160831,0.082415


In [18]:
sr_net_bench_dict = {
    'Momentum(10)':[0.1269, 0.1696],
    'Size-B/M(25)':[0.1477, 0.1702],
    'Industry(49)':[0.1512, 0.0816]
}
sr_net_bench_df = pd.DataFrame(sr_net_bench_dict, index=['1/N', 'W_g,t'])
sr_net_bench_df

,Momentum(10),Size-B/M(25),Industry(49)
1/N,0.1269,0.1477,0.1512
"W_g,t",0.1696,0.1702,0.0816


In [19]:
sr_net_diff_df = sr_net_df - sr_net_bench_df
sr_net_diff_df

,Momentum(10),Size-B/M(25),Industry(49)
1/N,0.000557,0.000014,0.000794
"W_g,t",-0.005044,-0.009369,0.000815


## 4.5 Performance agregation

In [14]:
naive_comb_performance = pd.concat([moment_naive_comb_performance, bm_naive_comb_performance, industry_naive_comb_performance], axis=1)
naive_comb_performance.columns = ['Momentum(10)', 'Size-B/M(25)', 'Industry(49)']
naive_comb_performance

,Momentum(10),Size-B/M(25),Industry(49)
Sharpe ratio,0.224982,0.203703,0.065447
Turnover,4.966185,24.605052,72.315573
Sharpe ratio net,0.168338,-0.003811,-0.162969


In [15]:
opt_comb_bench_dict = {
    'Momentum(10)':[0.2521, 2.0157, 0.2148],
    'Size-B/M(25)':[0.2479, 3.5212, 0.1707],
    'Industry(49)':[0.1194, 1.2583, 0.0697]
}
opt_comb_bench_df = pd.DataFrame(opt_comb_bench_dict, index=['Sharpe ratio', 'Turnover', 'Sharpe ratio net'])
opt_comb_bench_df

,Momentum(10),Size-B/M(25),Industry(49)
Sharpe ratio,0.2521,0.2479,0.1194
Turnover,2.0157,3.5212,1.2583
Sharpe ratio net,0.2148,0.1707,0.0697


In [16]:
naive_comb_bench_dict = {
    'Momentum(10)':[0.2375, 5.0765, 0.1814],
    'Size-B/M(25)':[0.1999, 54.3620, -0.0276],
    'Industry(49)':[0.0610, 91.2610, -0.1628]
}
naive_comb_bench_df = pd.DataFrame(naive_comb_bench_dict, index=['Sharpe ratio', 'Turnover', 'Sharpe ratio net'])
naive_comb_bench_df

,Momentum(10),Size-B/M(25),Industry(49)
Sharpe ratio,0.2375,0.1999,0.0610
Turnover,5.0765,54.3620,91.2610
Sharpe ratio net,0.1814,-0.0276,-0.1628


In [17]:
naive_comb_diff_df = naive_comb_performance - naive_comb_bench_df
naive_comb_diff_df

,Momentum(10),Size-B/M(25),Industry(49)
Sharpe ratio,-0.012518,0.003803,0.004447
Turnover,-0.110315,-29.756948,-18.945427
Sharpe ratio net,-0.013062,0.023789,-0.000169


## 4.5 output result to latex table

In [20]:
table_setting = {
    'df':naive_comb_performance,
    'table_name': 'naive combining portflolio performance',
    'label':'tbl:nav comb'
}
export_dataframe_to_latex_table(**table_setting)

write table to /Users/cheng/Dropbox/Apps/Overleaf/Weekly Report Cheng/table/naive combining portflolio performance.tex
